In [25]:
# Model design
import agentpy as ap

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython
import random
import time

In [26]:
# in starting points (13,0), the car should be able to also go to the another track
# in other words to (14,0), according to this grid this is the formost left lane

# in starting points (0,13), the car should be able to algo fo to the another track
# in other words (0,12) for example, accordint to this gris this is the foremost upway lane

# in starting points (12,24), the car should be able to also go to the another track
# in other words to (11,24), according to this grid this is the foremost right lane

#in starting points (24,14), the car should be able to also go to the another track
# in other words to (24,13), according to this grid this is the formost left lane

startingCoordinates = [(13,0),(0,11),(11,24),(24,13)]

movement1 = (0,1) # right
movement2 = (1,0) # down
movement3 = (0,-1) # left
movement4 = (-1,0) # up

stopLightPos = [(14,12),(12,10),(10,12),(12,14)]


In [27]:
class stopLight(ap.Agent):
    def setup(self):
        self.condition = 5  #5: green, 6:yellow, 7: red
    def conditionChange(self):
        if(self.condition==5):
            # self.condition=6
            # time.sleep(3)
            self.condition=7
        if(self.condition==7):
            self.condition=5

In [28]:
class streetIntersection(ap.Model):
    
    def setup(self):

        self.lightChanges = 10


        #define car agents
        self.carsl = ap.AgentList(self, self.p.n_cars)
        self.carsu = ap.AgentList(self, self.p.n_cars)
        self.carsr = ap.AgentList(self, self.p.n_cars)
        self.carsd = ap.AgentList(self, self.p.n_cars)

        self.stopLightl = ap.AgentList(self,1,stopLight)
        self.stopLightu = ap.AgentList(self,1,stopLight)
        self.stopLightr = ap.AgentList(self,1,stopLight)
        self.stopLightd = ap.AgentList(self,1,stopLight)
        



        #Create street grid
        self.street = ap.Grid(self,[self.p.size]*2,track_empty=True)
        #add each delimiter
        carspos1 = [startingCoordinates[0] for i in range(len(self.carsl))]
        carspos2 = [startingCoordinates[1] for i in range(len(self.carsl))]
        carspos3 = [startingCoordinates[2] for i in range(len(self.carsl))]
        carspos4 = [startingCoordinates[3] for i in range(len(self.carsl))]

        self.street.add_agents(self.carsl,positions=carspos1)
        self.street.add_agents(self.carsu,positions=carspos2)
        self.street.add_agents(self.carsr,positions=carspos3)
        self.street.add_agents(self.carsd,positions=carspos4)

        light1 = [stopLightPos[0] for i in range(len(self.stopLightl))]
        light2 = [stopLightPos[1] for i in range(len(self.stopLightl))]
        light3 = [stopLightPos[2] for i in range(len(self.stopLightl))]
        light4 = [stopLightPos[3] for i in range(len(self.stopLightl))]


        self.street.add_agents(self.stopLightl,positions=light1)
        self.street.add_agents(self.stopLightu,positions=light2)
        self.street.add_agents(self.stopLightr,positions=light3)
        self.street.add_agents(self.stopLightd,positions=light4)



        #CONDITIONS -> 2 - 4 different car IDs // 5:green light 6:yellow light 7:red light
        
        self.carsl.condition = 2
        self.carsu.condition = 1
        self.carsr.condition = 3
        self.carsd.condition = 4

    
    def step(self):
        #we need to select and separate cars in different conditions as they move differently

        # left cars
        moving_carsl = self.carsl.select(self.carsl.condition == 2)
        # up cars
        moving_carsu = self.carsu.select(self.carsu.condition == 1)
        # right cars
        moving_carsr = self.carsr.select(self.carsr.condition == 3)
        # down cars
        moving_carsd = self.carsd.select(self.carsd.condition == 4)

        


        # left cars
        bufferL = []

        for i,car in enumerate(moving_carsl):

            if(self.lightChanges==0):
                self.stopLightl.conditionChange()
                print(self.stopLightl.color)

                
            currentPos = self.street.positions[car] # get current position
            # movementL = movement1[1] # movement in the y axis of the car
            frontPos = (currentPos[0], currentPos[1] + 1) # position of the possible car in front
            frontPos2 = (currentPos[0], currentPos[1] + 2) # position of the possible car in front

            if frontPos[1] > 24 or frontPos2[1] > 24:
                break
 
            if (len(self.street.agents[frontPos].condition) == 0 and len(self.street.agents[frontPos2].condition) == 0):
                self.street.move_by(car, movement1) # move the car
                if (currentPos[1] == 22):
                    self.street.agents[currentPos].condition = None
                    # self.street.remove_agents(car)
        
        # up cars
        for i,car2 in enumerate(moving_carsu):

            currentPos = self.street.positions[car2] # get current position
            frontPos = (currentPos[0] + 1, currentPos[1]) # position of the possible car in front
            frontPos2 = (currentPos[0] + 2, currentPos[1]) # position of the possible car in front

            if frontPos[0] > 24 or frontPos2[0] > 24:
                break

            if (len(self.street.agents[frontPos].condition) == 0 and len(self.street.agents[frontPos2].condition) == 0):
                self.street.move_by(car2, movement2)
                if (currentPos[0] == 22):
                    self.street.agents[currentPos].condition = None
            
            

        # right cars
        for car3 in moving_carsr:

            currentPos = self.street.positions[car3] # get current position
            frontPos = (currentPos[0], currentPos[1] - 1) # position of the possible car in front
            frontPos2 = (currentPos[0], currentPos[1] - 2) # position of the possible car in front

            if frontPos[0] < 1 or frontPos2[0] < 1:
                break



            if (len(self.street.agents[frontPos].condition) == 0 and len(self.street.agents[frontPos2].condition) == 0):
                self.street.move_by(car3, movement3)
                if (currentPos[0] == 22):
                    self.street.agents[currentPos].condition = None
            
        
        
        # down cars
        for car4 in moving_carsd:

            currentPos = self.street.positions[car4]
            frontPos = (currentPos[0] - 1, currentPos[1])
            frontPos2 = (currentPos[0] - 2, currentPos[1])

            if frontPos[0] < 1 or frontPos2[0] < 1:
                break



            if (len(self.street.agents[frontPos].condition) == 0 and len(self.street.agents[frontPos2].condition) == 0):
                self.street.move_by(car4, movement4)
                if (currentPos[0] == 22):
                    self.street.agents[currentPos].condition = None
        
        if(self.lightChanges==0):
            self.lightChanges=10
        else:
            self.lightChanges-=1
        
                    
        

In [29]:
parameters = {
    'size': 25,
    'steps': 15,
    'n_cars': 10,
}

In [30]:
# Create single-run animation with custom colors

def animation_plot(model, ax):
    attr_grid = model.street.attr_grid('condition')
    color_dict = { 1:'#0000FF',2:'#FFFFFF', 3:'#FFCCCB', 4:'#33FFC4', 5:'#90EE90', 6:'#FFFFE0',7:'#FF5733', None: '#d5e5d5'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"TESTING STREET\n")

fig, ax = plt.subplots()
model = streetIntersection(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=12))

AttributeError: Agent (Obj 18) has no attribute 'color'.